# Stats Practice

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
import sys
print(sys.version)

3.7.7 (tags/v3.7.7:d7c567b08f, Mar 10 2020, 10:41:24) [MSC v.1900 64 bit (AMD64)]


### [예제 1] 지지율 여론 조사
선거 후보에 대해 지지율 조사를 하였다.  
조사 샘플 수는 1,000명이었고, A 후보 507명, B 후보 298명, 기타 195명이었다.  
두 후보의 지지율에 대해 95% 신뢰 구간을 추정하시오.

[참고 - 용어]
* 유의수준 : $\alpha$
* 신뢰수준 : $(1-\alpha)100\%$
* 표준편차 : $\sigma$
* 표준오차 : $\sigma\sqrt{n}$
* 표본오차 (허용오차, 오차범위) : $z\cdot\sigma\sqrt{n}$

In [9]:
# 지지율 데이터
n_A = 507
n_B = 298
n_X = 195
n_Tot = n_A + n_B + n_X

# 유의수준
alpha = 0.05 

# 표본비율 (Sampling Ratio) : p_hat = x/n
p_hat_A = n_A / n_Tot 
p_hat_B = n_B / n_Tot

# 확률변수(Random Variable)과 z값 (z-value, z-score)
rv_norm = stats.norm(0,1) # 표준 정규분포(Z=norm(0,1)) 사용
z_val = rv_norm.ppf(1-alpha/2)

# 한 모집단에서 여러 개의 모비율을 추정할 때, 동일한 최대허용 표본오차를 적용한다. (표본오차는 p_hat이 0.5일 때 최대가 된다.)
err_max = z_val*np.sqrt(0.5*0.5/n_Tot)

p_hat_A_l = p_hat_A - err_max
p_hat_A_r = p_hat_A + err_max
print(round(p_hat_A_l,2), ' <= p_hat_A <= ', round(p_hat_A_r,2), '[', round(p_hat_A*100,0), '+/-', round(err_max*100,0), ']')

p_hat_B_l = p_hat_B - err_max
p_hat_B_r = p_hat_B + err_max
print(round(p_hat_B_l,2), ' <= p_hat_B <= ', round(p_hat_B_r,2), '[', round(p_hat_B*100,0), '+/-', round(err_max*100,0), ']')

0.48  <= p_hat_A <=  0.54 [ 51.0 +/- 3.0 ]
0.27  <= p_hat_B <=  0.33 [ 30.0 +/- 3.0 ]


### [예제 2] 연령대별 지지 정당
1,000명을 샘플 조사하여 연령대별 지지 정당에 대하여 다음의 표를 만들었다.  
두 변수(연령대와 지지 정당)가 관계가 있는지 독립성 검정을 하시오.

In [165]:
crosstab = pd.DataFrame([[45, 90, 95], [50, 120, 100], [50, 120, 70], [140, 50, 70]], \
                        index=['20대', '30대', '40대', '50대 이상'], columns=['A정당', 'B정당', 'C정당'])
crosstab

,A정당,B정당,C정당
20대,45,90,95
30대,50,120,100
40대,50,120,70
50대 이상,140,50,70


In [166]:
# 가설을 세운다
# H0 : 두 변수는 관계사 없다. (연관 계수 = 0)  
# H1 : 두 변수는 관계가 있다. (연관 계수 <>0, 양측 검정)

# 독립성 검정 통계량 chi^2를 구한다
# chi^2 = sum((관측도수-기대도수)^2/기대도수)

# 표(교차 집계표)에 연령대 합계와 정당 합계를 추가한다
crosstab['연령대 합계'] = crosstab.sum(axis=1)
crosstab.loc['정당 합계'] = crosstab.sum(axis=0)

In [167]:
crosstab

,A정당,B정당,C정당,연령대 합계
20대,45,90,95,230
30대,50,120,100,270
40대,50,120,70,240
50대 이상,140,50,70,260
정당 합계,285,380,335,1000


In [168]:
# 기대도수를 계산한다
exp_tab = crosstab # 복사
exp_tab = exp_tab.astype('float') # 데이터 타입 변경

def calcExpFreq(crosstab, exp_tab):
    idx_col_sum = crosstab.shape[1] - 1
    idx_row_sum = crosstab.shape[0] - 1
    for irow in range(0, idx_row_sum):
        for icol in range(0, idx_col_sum):
            exp_tab.iat[irow, icol] = crosstab.iat[irow, idx_col_sum] * crosstab.iat[idx_row_sum, icol] \
                                        / crosstab.iat[idx_row_sum, idx_col_sum]

calcExpFreq(crosstab, exp_tab)

exp_tab

,A정당,B정당,C정당,연령대 합계
20대,65.55,87.4,77.05,230.0
30대,76.95,102.6,90.45,270.0
40대,68.40,91.2,80.40,240.0
50대 이상,74.10,98.8,87.10,260.0
정당 합계,285.00,380.0,335.00,1000.0


In [169]:
# chi^2를 계산한다
# chi^2 = (관측도수-기대도수)^2/기대도수
chisq_tab = exp_tab # 복사

idx_col_sum = crosstab.shape[1] - 1
idx_row_sum = crosstab.shape[0] - 1
def calcChiSq(crosstab, exp_tab, chisq_tab):
    for irow in range(0, idx_row_sum):
        for icol in range(0, idx_col_sum):
            chisq_tab.iat[irow, icol] = (crosstab.iat[irow, icol] - exp_tab.iat[irow, icol])**2 \
                                        / exp_tab.iat[irow, icol]

calcChiSq(crosstab, exp_tab, chisq_tab)

# 표(교차 집계표)에 연령대 합계와 정당 합계를 다시 계산한다
chisq_tab['연령대 합계'] = chisq_tab[['A정당', 'B정당', 'C정당']].sum(axis=1)
chisq_tab.loc['정당 합계'] = chisq_tab.iloc[0:idx_row_sum,:].sum(axis=0)

chisq_tab

,A정당,B정당,C정당,연령대 합계
20대,6.442449,0.077346,4.181733,10.701527
30대,9.438629,2.950877,1.008320,13.397826
40대,4.949708,9.094737,1.345274,15.389718
50대 이상,58.607422,24.103644,3.357176,86.068242
정당 합계,79.438207,36.226603,9.892501,125.557312


In [170]:
chisq_0 = chisq_tab.iat[idx_row_sum, idx_col_sum]

# 확률변수(Random Variable)과 임계치를 구한다
rv_chi2 = stats.chi2(6) # 카이제곱분포의 자유도 = (행-1)*(열-1)
chisq_c = rv_chi2.ppf(1-alpha/2) # chi2.ppf (percent point function) : 분포의 좌측 누적 확률을 인자로 하여 임계치(기각역)를 구한다

chisq_c

14.44937533544792

In [171]:
# chisq_0가 기각역에 포함되면 H0를 기각한다
# 그렇지 않으면 H0를 채택한다 (H0가 틀렸다고는 할 수 없다)
if (chisq_c <= chisq_0):
    print('chisq_c:', round(chisq_c,2), ' <= ', 'chisq_0:', round(chisq_0,2), '[H0 Rejected]')
else:
    print('chisq_c:', round(chisq_c,2), ' > ', 'chisq_0:', round(chisq_0,2), '[H0 Accepted]')

chisq_c: 14.45  <=  chisq_0: 125.56 [H0 Rejected]


In [172]:
# 결론 : 두 변수는 관계가 있다.

# 연관 계수를 구해 보자
# 연관 계수 V = sqrt(chisq_0^2 / N*min(행-1,열-1))
round(np.sqrt(chisq_0 / (1000*2)),2)

0.25

In [173]:
# 독립성 검정 결과는 '두 변수가 관계가 있다'로 나왔는데 .. 
# 연관 계수는 0.25로 '다소 약하게 관련되어 있다'로 나타나서
# 좀 의아한 생각이 든다 ..
# 아래에 다른 예제를 살펴 본다

### [예제 3] 연령대별 야행 선호지
100명을 샘플 조사하여 연령대별 여행 선호지에 대하여 다음의 표를 만들었다.  
두 변수(연령대와 여행 선호지)가 관계가 있는지 독립성 검정을 하시오.

In [174]:
crosstab = pd.DataFrame([[9, 5, 6], [14, 11, 5], [10, 13, 7], [5, 6, 9]], \
                        index=['20대', '30대', '40대', '50대 이상'], columns=['북미', '유럽', '아시아'])
crosstab

,북미,유럽,아시아
20대,9,5,6
30대,14,11,5
40대,10,13,7
50대 이상,5,6,9


In [175]:
# 가설을 세운다
# H0 : 두 변수는 관계사 없다. (연관 계수 = 0)  
# H1 : 두 변수는 관계가 있다. (연관 계수 <>0, 양측 검정)

# 독립성 검정 통계량 chi^2를 구한다
# chi^2 = sum((관측도수-기대도수)^2/기대도수)

# 표(교차 집계표)에 연령대 합계와 정당 합계를 추가한다
crosstab['연령대 합계'] = crosstab.sum(axis=1)
crosstab.loc['여행지 합계'] = crosstab.sum(axis=0)

In [176]:
crosstab

,북미,유럽,아시아,연령대 합계
20대,9,5,6,20
30대,14,11,5,30
40대,10,13,7,30
50대 이상,5,6,9,20
여행지 합계,38,35,27,100


In [177]:
# 기대도수를 계산한다
exp_tab = crosstab # 복사
exp_tab = exp_tab.astype('float') # 데이터 타입 변경

def calcExpFreq(crosstab, exp_tab):
    idx_col_sum = crosstab.shape[1] - 1
    idx_row_sum = crosstab.shape[0] - 1
    for irow in range(0, idx_row_sum):
        for icol in range(0, idx_col_sum):
            exp_tab.iat[irow, icol] = crosstab.iat[irow, idx_col_sum] * crosstab.iat[idx_row_sum, icol] \
                                        / crosstab.iat[idx_row_sum, idx_col_sum]

calcExpFreq(crosstab, exp_tab)

exp_tab

,북미,유럽,아시아,연령대 합계
20대,7.6,7.0,5.4,20.0
30대,11.4,10.5,8.1,30.0
40대,11.4,10.5,8.1,30.0
50대 이상,7.6,7.0,5.4,20.0
여행지 합계,38.0,35.0,27.0,100.0


In [178]:
# chi^2를 계산한다
# chi^2 = (관측도수-기대도수)^2/기대도수
chisq_tab = exp_tab # 복사

idx_col_sum = crosstab.shape[1] - 1
idx_row_sum = crosstab.shape[0] - 1
def calcChiSq(crosstab, exp_tab, chisq_tab):
    for irow in range(0, idx_row_sum):
        for icol in range(0, idx_col_sum):
            chisq_tab.iat[irow, icol] = (crosstab.iat[irow, icol] - exp_tab.iat[irow, icol])**2 \
                                        / exp_tab.iat[irow, icol]

calcChiSq(crosstab, exp_tab, chisq_tab)

# 표(교차 집계표)에 연령대 합계와 정당 합계를 다시 계산한다
chisq_tab['연령대 합계'] = chisq_tab[['북미', '유럽', '아시아']].sum(axis=1)
chisq_tab.loc['여행지 합계'] = chisq_tab.iloc[0:idx_row_sum,:].sum(axis=0)

chisq_tab

,북미,유럽,아시아,연령대 합계
20대,0.257895,0.571429,0.066667,0.895990
30대,0.592982,0.023810,1.186420,1.803212
40대,0.171930,0.595238,0.149383,0.916551
50대 이상,0.889474,0.142857,2.400000,3.432331
여행지 합계,1.912281,1.333333,3.802469,7.048083


In [179]:
chisq_0 = chisq_tab.iat[idx_row_sum, idx_col_sum]

# 확률변수(Random Variable)과 임계치를 구한다
rv_chi2 = stats.chi2(6) # 카이제곱분포의 자유도 = (행-1)*(열-1)
chisq_c = rv_chi2.ppf(1-alpha/2) # chi2.ppf (percent point function) : 분포의 좌측 누적 확률을 인자로 하여 임계치(기각역)를 구한다

chisq_c

14.44937533544792

In [180]:
# chisq_0가 기각역에 포함되면 H0를 기각한다
# 그렇지 않으면 H0를 채택한다 (H0가 틀렸다고는 할 수 없다)
if (chisq_c <= chisq_0):
    print('chisq_c:', round(chisq_c,2), ' <= ', 'chisq_0:', round(chisq_0,2), '[H0 Rejected]')
else:
    print('chisq_c:', round(chisq_c,2), ' > ', 'chisq_0:', round(chisq_0,2), '[H0 Accepted]')

chisq_c: 14.45  >  chisq_0: 7.05 [H0 Accepted]


In [181]:
# 결론 : 두 변수는 관계가 없다.

# 연관 계수를 구해 보자
# 연관 계수 V = sqrt(chisq_0^2 / N*min(행-1,열-1))
round(np.sqrt(chisq_0 / (100*2)),2)

0.19

In [182]:
# 이 예제에서는 독립성 검정 결과도 '두 변수가 관계가 없다'로 나오고
# 연관 계수도 0.19로 '매우 약하게 관련되어 있다(즉 관련이 없다)'로 나타나서
# 두 가지가 일치하고 있다